# Peak Finding Exercise

When dealing with a series of values, we might commonly want to know what the maximum value is. For example, if we were looking at the temperature of a sample in a chemistry lab, we might want to know what the maximum temperature was during the course of an experiment. We might have some data which looks something like this:

<center><img src="../Resources/peak_full.png" style="height:300px" /></center>

In this case, the data has been taken every 1 hour, however, its clear that this is not often enough to capture all the variation of the actual temperature. For example, we can zoom in on the time period between 3 and 7 hours and plot the actual temperature alongside the data points recorded:

<center><img src="../Resources/peak_closeup.png" style="height:300px" /></center>

As we can see, the data points do not capture the full variation of the temperature over time. In particular, the data points do not capture the peak temperature or the timing of this peak. In some experiments ot simulations, capturing these features of a data set can be very useful.

In this exercise, you will build a code which can interpolate between points of a data set to find the peak value and the time at which this peak occurs. 

## The Quadratic Approximation

We can approximate the shape of a curve around a point by defining a quadratic function which passes through the points an the points on either side of it. For the peak we're discussing, this function may look like this:c

<center><img src="../Resources/peak_fit.png" style="height:300px" /></center>

As we can see, the fit still isn't perfect, but we are getting an improved estimate of the peak value and the time at which this peak occurs. The quadratic fit has the equation:

$$ y = ax^{2} + bx + c $$

where $a$, $b$ and $c$ are the quadratic, linear, and constant coefficients of the function. We can find these coefficients by solving the following equations:

$$ a = \frac{x_{1}(y_{3} - y_{2}) + x_{2}(y_{1} - y_{3}) + x_{3}(y_{2} - y_{1})}{(x_{1} - x_{2})(x_{1} - x_{3})(x_{2} - x_{3})} $$

$$ b = \frac{y_{2} - y_{1}}{x_{2} - x{1}} - a (x_{1} + x_{2})$$

$$ c = y_{1} - ax_{1} ^{2} - bx_{1} $$

where $(x_{1}, y_{1})$, $(x_{2}, y_{2})$ and $(x_{3}, y_{3})$ are the Cartesian coordinates of the point to the left, the central point, and the point to the right of the region we're approximating.

### Task 1

Write a function in the code cell below named ```get_quadratic_coefficients``` which receives 6 arguments. These arguments should be, in order, $x_{1}$, $y_{1}$, $x_{2}$, $y_{2}$, $x_{3}$, and $y_{3}$. The function should return a list with three entries, containing the values of $a$, $b$, and $c$. There are a few calls to the function in the cell below, which you can use to test your function.

In [ ]:
# Write your function here







# Test Cases
print(get_quadratic_coefficients(-1, 0, 0, 0, 1, 0)) # Should return [0.0, 0.0, 0.0]

print(get_quadratic_coefficients(4.2, -10.1, 4.4, -9.5, 4.6, -8.9)) # Should return [0.0, 3.0, -22.7]

print(get_quadratic_coefficients(-10, 200, 0, 0, 10, 200)) # Should return [2.0, 0.0, 0.0]

print(get_quadratic_coefficients(4, 10, 5, 20, 6, 18)) # Should return [-6.0, 64.0, -150.0]

For each of the exercises in this notebook, sample solutions can be found in ```Sample Solutions/Sample Solutions 7 - Peak Finding.ipynb```.

## Finding the Peak

Once we have the coefficients of the quadratic function, we can find the peak by finding the $x$ value at which the function reaches its maximum. To do this, we can find the value of $x$ at which the gradient of the function is equal to zero. The gradient of the function is given by:

$$ g = 2ax + b $$

and is equal to zero when:

$$ x_{flat} = -\frac{b}{2a} $$

If the value of $a$ is positive, then this value of $x_{flat}$ corresponds to a minimum, and if the value of $a$ is negative, then this value of $x_{flat}$ corresponds to a maximum. If the value of $a$ is zero, then the quadratic coefficient is zero and the function has no peak.

### Task 2

Write a function named ```get_peak_in_region``` which finds the peak of the function approximated by three data-points. This function should receive 6 arguments. These arguments should be, in order, $x_{1}$, $y_{1}$, $x_{2}$, $y_{2}$, $x_{3}$, and $y_{3}$, where $(x_{1}, y_{1})$, $(x_{2}, y_{2})$ and $(x_{3}, y_{3})$ are the Cartesian coordinates of the point on the left boundary left, the central point, and the point on the right boundary of the region we're approximating.

This function should use your function ```get_quadratic_coefficients``` to find the coefficients of the quadratic function which approximates the data. It should then use these coefficients to find the $x$ value at which the gradient of the function is equal to zero. 

If this value of $x$ is between $x_{1}$ and $x_{3}$ inclusive (i.e. it's in the region we're approximating) AND the value of $a$ is negative (i.e. our approximation contains a peak) then the function should return a list containing the values of $x$ at this peak and the value of $y$ at this point (i.e the Cartesian coordinates of the peak).

If this value of $x$ is not between $x_{1}$ and $x_{3}$, or if the approximation is flat or contains a minimum, then the function should return ```None```.

When writing this function, consider if there are any useful ways you could split up the functionality of your function into other functions which could be useful elsewhere, or be independently tested.

In [ ]:







# Test Cases
print(get_peak_in_region(0.8, 0, 1, 1, 1.2, 0)) # Should return [1.0, 1.0]

print(get_peak_in_region(3, -9, 4, -9, 5, -12)) # Should return [3.5, -8.625]

print(get_peak_in_region(5, 1.2, 5.1, 1.3, 5.2, 1.4)) # Should return None (the approximation should be straight)

print(get_peak_in_region(1, 1, 2, 0, 3, -2)) # Should be None (the peak is to the before the region we're approximating)

print(get_peak_in_region(-10, 0, -5, 10, 0, 15)) # Should be None (the peak is after the region we're approximating)

print(get_peak_in_region(0, 2, 1, 1, 2, 2)) # Should be None (the flat section of the approximation is a trough not a peak)

## Examining a Whole Dataset

Now that we have a function which can determine if there is a peak in a region of three data points and where that peak is, we can use this function to examine a whole dataset and find which is the highest peak.

To do this, we will consider each region of three data points in turn, and use our function ```get_peak_in_region``` to determine if there is a value in each region. If there is, we will evaluate if it higher than the highest peak we have found so far. If it is, we will store the coordinates of this peak.

For example, if we have data points $(x_{1}, y_{1}), (x_{2}, y_{2}), ... , (x_{10}, y_{10})$ We will first examine the region between $x_{1}$ and $x_{3}$, then the region between $x_{2}$ and $x_{4}$, then the region between $x_{3}$ and $x_{5}$, and so on.

### Task 3

Write a function named ```get_peak_data_set```. This should receive two arguments. The first is a list containing the x values of the data set, and the second is a list containing the y values of the data set. The function should return a list containing the Cartesian coordinates of the highest peak in the data set. If the dataset does not contain any peaks, the function should return ```None```.

You may assume that the two lists provided as arguments will be the same length, will contain only numerical values, and that the data set will contain at least three points.

Write your function in the cell below. There are some calls to the function in the cell which you can use to test your function.